# Create and train MitoSplit-Net model

## Import required Python libraries



In [1]:
from IPython import get_ipython
ipython = get_ipython()
if ipython:
    ipython.magic("reload_ext autoreload")
    ipython.magic("autoreload 2")


import util
import plotting
import training

import numpy as np
import matplotlib.pyplot as plt
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('axes', labelsize=20)
plt.rc('legend', fontsize=18)
from tqdm import tqdm

import tensorflow as tf

/tmp/ipykernel_3653956/1041950970.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("reload_ext autoreload")
/tmp/ipykernel_3653956/1041950970.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")
2025-01-01 21:40:26.040412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735764026.072716 3653956 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735764026.082651 3653956 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-01 21:

In [2]:
#Define GPU device where the code will run on
gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/')

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


I0000 00:00:1735764033.771208 3653956 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9540 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1


## Data and models directories

In [3]:
base_dir = '/mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/'
print('base_dir:', base_dir)

data_path = base_dir+'Data/'
print('data_path:', data_path)

model_path = base_dir+'Models/'
print('model_path:', model_path)

base_dir: /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/
data_path: /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/
model_path: /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/


## Create model, split dataset and train

### No preprocessing, different batch sizes

#### Only mito channel as input

In [4]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)

#Outputs
output_data = util.load_h5(data_path, 'Proc')
print('Outputs:', output_data.shape)


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Proc.h5
Converting to array
Outputs: (37000, 128, 128)


In [5]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=b) 

Model: ref_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *


2025-01-01 21:41:12.538820: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 387973120 exceeds 10% of free system memory.
2025-01-01 21:41:14.190916: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 387973120 exceeds 10% of free system memory.
2025-01-01 21:41:14.687149: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 387973120 exceeds 10% of free system memory.
2025-01-01 21:41:15.089112: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 387973120 exceeds 10% of free system memory.


Epoch 1/20


I0000 00:00:1735764084.430003 3654302 service.cc:148] XLA service 0x7f0d80001540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735764084.430065 3654302 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2025-01-01 21:41:24.699719: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735764085.871277 3654302 cuda_dnn.cc:529] Loaded cuDNN version 90600
2025-01-01 21:41:26.775413: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[8,16,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,8,64,64]{3,2,1,0}, f32[16,8,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{

2960/2960 - 61s - 21ms/step - binary_accuracy: 0.9661 - loss: -2.1516e-01 - val_binary_accuracy: 0.9771 - val_loss: -4.6951e-01
Epoch 2/20
2960/2960 - 42s - 14ms/step - binary_accuracy: 0.9831 - loss: -5.3934e-01 - val_binary_accuracy: 0.9804 - val_loss: -5.1707e-01
Epoch 3/20
2960/2960 - 42s - 14ms/step - binary_accuracy: 0.9866 - loss: -6.1487e-01 - val_binary_accuracy: 0.9917 - val_loss: -6.2194e-01
Epoch 4/20
2960/2960 - 42s - 14ms/step - binary_accuracy: 0.9876 - loss: -6.4105e-01 - val_binary_accuracy: 0.9911 - val_loss: -6.6452e-01
Epoch 5/20
2960/2960 - 41s - 14ms/step - binary_accuracy: 0.9890 - loss: -6.7220e-01 - val_binary_accuracy: 0.9869 - val_loss: -6.2991e-01
Epoch 6/20
2960/2960 - 41s - 14ms/step - binary_accuracy: 0.9898 - loss: -6.8786e-01 - val_binary_accuracy: 0.9882 - val_loss: -6.8566e-01
Epoch 7/20
2960/2960 - 41s - 14ms/step - binary_accuracy: 0.9903 - loss: -6.9870e-01 - val_binary_accuracy: 0.9906 - val_loss: -6.9183e-01
Epoch 8/20
2960/2960 - 41s - 14ms/step

...

1480/1480 - 49s - 33ms/step - binary_accuracy: 0.9691 - loss: -1.9916e-01 - val_binary_accuracy: 0.9569 - val_loss: -3.4202e-01
Epoch 2/20
1480/1480 - 31s - 21ms/step - binary_accuracy: 0.9817 - loss: -5.0125e-01 - val_binary_accuracy: 0.9893 - val_loss: -4.8857e-01
Epoch 3/20
1480/1480 - 30s - 20ms/step - binary_accuracy: 0.9852 - loss: -5.8490e-01 - val_binary_accuracy: 0.9885 - val_loss: -5.9152e-01
Epoch 4/20
1480/1480 - 30s - 20ms/step - binary_accuracy: 0.9868 - loss: -6.2551e-01 - val_binary_accuracy: 0.9889 - val_loss: -6.3476e-01
Epoch 5/20
1480/1480 - 30s - 20ms/step - binary_accuracy: 0.9881 - loss: -6.5586e-01 - val_binary_accuracy: 0.9882 - val_loss: -6.4366e-01
Epoch 6/20
1480/1480 - 43s - 29ms/step - binary_accuracy: 0.9889 - loss: -6.7316e-01 - val_binary_accuracy: 0.9884 - val_loss: -6.5880e-01
Epoch 7/20
1480/1480 - 30s - 20ms/step - binary_accuracy: 0.9896 - loss: -6.8715e-01 - val_binary_accuracy: 0.9922 - val_loss: -6.5119e-01
Epoch 8/20
1480/1480 - 29s - 20ms/step

2025-01-01 22:06:08.943334: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[32,16,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,8,64,64]{3,2,1,0}, f32[16,8,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-01 22:06:08.996287: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[32,16,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,16,64,64]{3,2,1,0}, f32[16,16,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", b

740/740 - 45s - 61ms/step - binary_accuracy: 0.9739 - loss: -1.3362e-01 - val_binary_accuracy: 0.9775 - val_loss: -3.7229e-01
Epoch 2/20
740/740 - 24s - 33ms/step - binary_accuracy: 0.9813 - loss: -4.7511e-01 - val_binary_accuracy: 0.9869 - val_loss: -4.6698e-01
Epoch 3/20
740/740 - 24s - 33ms/step - binary_accuracy: 0.9846 - loss: -5.7456e-01 - val_binary_accuracy: 0.9758 - val_loss: -5.4731e-01
Epoch 4/20
740/740 - 24s - 33ms/step - binary_accuracy: 0.9862 - loss: -6.1836e-01 - val_binary_accuracy: 0.9849 - val_loss: -6.0929e-01
Epoch 5/20
740/740 - 24s - 33ms/step - binary_accuracy: 0.9876 - loss: -6.5060e-01 - val_binary_accuracy: 0.9885 - val_loss: -6.3372e-01
Epoch 6/20
740/740 - 25s - 33ms/step - binary_accuracy: 0.9886 - loss: -6.7414e-01 - val_binary_accuracy: 0.9892 - val_loss: -6.3949e-01
Epoch 7/20
740/740 - 41s - 55ms/step - binary_accuracy: 0.9894 - loss: -6.9099e-01 - val_binary_accuracy: 0.9892 - val_loss: -6.7467e-01
Epoch 8/20
740/740 - 41s - 55ms/step - binary_accura

2025-01-01 22:15:32.237626: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 10.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-01 22:15:32.237705: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 10.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-01 22:15:33.268415: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{} for conv (f32[256,16,64,64]{3,2,1,0}, u8[0]{0}) custom-call(f32[256,8,64,64]{3,2,1,0}, f32[16,8,3,3]{3,2,1,0}, f32[16]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBia

93/93 - 79s - 847ms/step - binary_accuracy: 0.6480 - loss: 0.6400 - val_binary_accuracy: 0.8661 - val_loss: 0.6526
Epoch 2/20
93/93 - 19s - 206ms/step - binary_accuracy: 0.9008 - loss: 0.2869 - val_binary_accuracy: 0.9772 - val_loss: 0.4649
Epoch 3/20
93/93 - 19s - 199ms/step - binary_accuracy: 0.9513 - loss: 0.0461 - val_binary_accuracy: 0.9866 - val_loss: 0.2713
Epoch 4/20
93/93 - 18s - 196ms/step - binary_accuracy: 0.9693 - loss: -9.2944e-02 - val_binary_accuracy: 0.9857 - val_loss: 0.0099
Epoch 5/20
93/93 - 18s - 197ms/step - binary_accuracy: 0.9754 - loss: -1.9297e-01 - val_binary_accuracy: 0.9904 - val_loss: -1.9260e-03
Epoch 6/20
93/93 - 18s - 197ms/step - binary_accuracy: 0.9807 - loss: -2.6798e-01 - val_binary_accuracy: 0.9869 - val_loss: -2.5562e-01
Epoch 7/20
93/93 - 20s - 220ms/step - binary_accuracy: 0.9840 - loss: -3.2655e-01 - val_binary_accuracy: 0.9827 - val_loss: -2.8804e-01
Epoch 8/20
93/93 - 18s - 196ms/step - binary_accuracy: 0.9847 - loss: -3.6936e-01 - val_binary

In [6]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b8/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b16/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b32/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b256/model.h5
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b8/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b16/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b32/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b256/history
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b8/frames_test

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b16/frames_test

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/ref_f8_c9_b32/frames_test

Saving /mnt/LE

#### Mito + Drp1 channels as inputs

In [7]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
input_data = np.stack((input_data, util.load_h5(data_path, 'Drp1')), axis=-1)
print('Inputs'+':', input_data.shape)

#Outputs
output_data = util.load_h5(data_path, 'Proc')
print('Outputs:', output_data.shape)


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Mito.h5
Converting to array

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Drp1.h5
Converting to array
Inputs: (37000, 128, 128, 2)

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Proc.h5
Converting to array
Outputs: (37000, 128, 128)


In [8]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  nb_input_channels = 2
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'multich_ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize, nb_input_channels)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=b) 

Model: multich_ref_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
2960/2960 - 62s - 21ms/step - binary_accuracy: 0.9672 - loss: -5.4430e-01 - val_binary_accuracy: 0.9937 - val_loss: -7.0498e-01
Epoch 2/20
2960/2960 - 44s - 15ms/step - binary_accuracy: 0.9907 - loss: -7.7461e-01 - val_binary_accuracy: 0.9932 - val_loss: -7.5730e-01
Epoch 3/20
2960/2960 - 43s - 15ms/step - binary_accuracy: 0.9917 - loss: -7.9718e-01 - val_binary_accuracy: 0.9918 - val_loss: -7.6903e-01
Epoch 4/20
2960/2960 - 44s - 15ms/step - binary_accuracy: 0.9922 - loss: -8.1041e-01 - val_binary_accuracy: 0.9944 - val_loss: -7.7171e-01
Epoch 5/20
2960/2960 - 44s - 15ms/step - binary_accuracy: 0.9926 - loss: -8.1877e-01 - val_binary_accuracy: 0.9940 - val_loss: -7.7705e-01
Epoch 6/20
2960/2960 - 43s - 15ms/step - binary_accuracy: 0.9929 - loss: -8.2394e-01 - val_binary_accuracy: 0.9913 - val_loss: -7.8271e-01
Epoch 7/20
2960/2960 - 43s - 15ms/step - binary_accuracy: 0.9

In [9]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b8/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b16/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b32/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b256/model.h5
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b8/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b16/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b32/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b256/history
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b8/frames_test

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_ref_f8_c9_b16/frames_test

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSp

### Mito & WatProc, different gaussian dilations

#### Only mito channel as input

In [10]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)


In [11]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = 16
  
  optimal_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  model, history, frames_test = {}, {}, {}
  
  for s, t in zip(optimal_sigma, threshold):
    model_name = 'wp_f%i_c%i_b%i_s%.1f_t%.i'%(nb_filters, firstConvSize, batch_size, s, t)
    print('Model:', model_name)
    #Outputs
    output_data = util.load_h5(data_path, 'WatProc_s%.1f_t%.i'%(s, t))
    print('Outputs:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_optimal_sigma

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_intensity_threshold
Model: wp_f8_c9_b16_s1.9_t1

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/WatProc_s1.9_t1.h5
Converting to array
Outputs: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 48s - 32ms/step - binary_accuracy: 0.9961 - loss: 0.0696 - val_binary_accuracy: 0.9968 - val_loss: 0.0112
Epoch 2/20
1480/1480 - 29s - 20ms/step - binary_accuracy: 0.9967 - loss: 0.0064 - val_binary_accuracy: 0.9968 - val_loss: 0.0039
Epoch 3/20
1480/1480 - 29s - 20ms/step - binary_accuracy: 0.9967 - loss: 0.0031 - val_binary_accuracy: 0.9968 - val_loss: 0.0024
Epoch 4/20
1480/1480 - 29s - 20ms/step - binary_accuracy: 0.9967 - loss: 0.0022 - val_binary_accuracy: 0.9968

In [12]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s1.9_t1/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s1.9_t6/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.0_t21/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.1_t33/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.1_t48/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.2_t68/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.2_t98/model.h5
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s1.9_t1/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s1.9_t6/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.0_t21/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.1_t33/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/wp_f8_c9_b16_s2.1_t48/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/

In [13]:
del input_data, output_data

#### Mito + Drp1 channels as inputs

In [14]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
input_data = np.stack((input_data, util.load_h5(data_path, 'Drp1')), axis=-1)
print('Inputs'+':', input_data.shape)


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Mito.h5
Converting to array

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Drp1.h5
Converting to array
Inputs: (37000, 128, 128, 2)


In [15]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  nb_input_channels = 2
  batch_size = 16
  
  optimal_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  model, history, frames_test = {}, {}, {}
  
  for s, t in zip(optimal_sigma, threshold):
    model_name = 'multich_wp_f%i_c%i_b%i_s%.1f_t%.i'%(nb_filters, firstConvSize, batch_size, s, t)
    print('Model:', model_name)
    #Outputs
    output_data = util.load_h5(data_path, 'WatProc_s%.1f_t%.i'%(s, t))
    print('Outputs:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize, nb_input_channels)
    
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_optimal_sigma

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_intensity_threshold
Model: multich_wp_f8_c9_b16_s1.9_t1

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/WatProc_s1.9_t1.h5
Converting to array
Outputs: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 52s - 35ms/step - binary_accuracy: 0.9926 - loss: 0.1171 - val_binary_accuracy: 0.9968 - val_loss: 0.0191
Epoch 2/20
1480/1480 - 34s - 23ms/step - binary_accuracy: 0.9967 - loss: 0.0093 - val_binary_accuracy: 0.9968 - val_loss: 0.0046
Epoch 3/20
1480/1480 - 34s - 23ms/step - binary_accuracy: 0.9967 - loss: 0.0033 - val_binary_accuracy: 0.9968 - val_loss: 0.0023
Epoch 4/20
1480/1480 - 34s - 23ms/step - binary_accuracy: 0.9967 - loss: 0.0019 - val_binary_accuracy

In [16]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s1.9_t1/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s1.9_t6/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.0_t21/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.1_t33/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.1_t48/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.2_t68/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.2_t98/model.h5
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s1.9_t1/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s1.9_t6/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.0_t21/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.1_t33/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_wp_f8_c9_b16_s2.1_t48/history

Saving /mnt/LEB/Scientific_projects/deep_

In [17]:
del input_data, output_data

## Spatiotemporal gaussian filter

### Only mito channel as input

In [18]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)


In [19]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = 16
  
  dilation_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  dilation_threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  event_score_gaussian_threshold = util.load_pkl(data_path, 'event_score_gaussian_thresholds')
  time_window_size = 5
  
  model, history, frames_test = {}, {}, {}
  
  for sigma, threshold, gaussian_threshold in zip(dilation_sigma, dilation_threshold, event_score_gaussian_threshold):
    metadata = 's%.1f_t%i_w%i_gt%i'%(sigma, threshold, time_window_size, gaussian_threshold)
    model_name = 'spatemp_wp_f%i_c%i_b%i_'%(nb_filters, firstConvSize, batch_size) + metadata
    print('Model:', model_name)

    #Output
    output_data = util.load_h5(data_path, 'spatemp_WatProc_'+metadata)
    print('Output:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize)

    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 
    del output_data


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_optimal_sigma

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_intensity_threshold

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/event_score_gaussian_thresholds
Model: spatemp_wp_f8_c9_b16_s1.9_t1_w5_gt38

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/spatemp_WatProc_s1.9_t1_w5_gt38.h5
Converting to array
Output: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 52s - 35ms/step - binary_accuracy: 0.9971 - loss: 0.0860 - val_binary_accuracy: 0.9982 - val_loss: 0.0138
Epoch 2/20
1480/1480 - 32s - 22ms/step - binary_accuracy: 0.9982 - loss: 0.0070 - val_binary_accuracy: 0.9982 - val_loss: 0.0039
Epoch 3/20
1480/1480 - 32s - 22ms/step - binary_accuracy: 0.9982 - loss: 0.003

In [20]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s1.9_t1_w5_gt38/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.1_t48_w5_gt91/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.2_t68_w5_gt109/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.2_t98_w5_gt127/model.h5
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s1.9_t1_w5_gt38/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.1_t48_w5_gt91/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/spatemp_wp_f8_c9_b16_s2.2_t68_w5

### Mito + Drp1 channels as inputs

In [21]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
input_data = np.stack((input_data, util.load_h5(data_path, 'Drp1')), axis=-1)
print('Inputs'+':', input_data.shape)


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Mito.h5
Converting to array

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/Drp1.h5
Converting to array
Inputs: (37000, 128, 128, 2)


In [22]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  nb_input_channels = 2
  batch_size = 16
  
  dilation_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  dilation_threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  event_score_gaussian_threshold = util.load_pkl(data_path, 'event_score_gaussian_thresholds')
  time_window_size = 5
  
  model, history, frames_test = {}, {}, {}
  
  for sigma, threshold, gaussian_threshold in zip(dilation_sigma, dilation_threshold, event_score_gaussian_threshold):
    metadata = 's%.1f_t%i_w%i_gt%i'%(sigma, threshold, time_window_size, gaussian_threshold)
    model_name = 'multich_spatemp_wp_f%i_c%i_b%i_'%(nb_filters, firstConvSize, batch_size) + metadata
    print('Model:', model_name)

    #Output
    output_data = util.load_h5(data_path, 'spatemp_WatProc_'+metadata)
    print('Output:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize, nb_input_channels)

    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 
    del output_data


Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_optimal_sigma

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/max_intensity_threshold

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/event_score_gaussian_thresholds
Model: multich_spatemp_wp_f8_c9_b16_s1.9_t1_w5_gt38

Loading /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Data/spatemp_WatProc_s1.9_t1_w5_gt38.h5
Converting to array
Output: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 51s - 34ms/step - binary_accuracy: 0.9483 - loss: 0.4407 - val_binary_accuracy: 0.9982 - val_loss: 0.1530
Epoch 2/20
1480/1480 - 34s - 23ms/step - binary_accuracy: 0.9982 - loss: 0.0984 - val_binary_accuracy: 0.9982 - val_loss: 0.0244
Epoch 3/20
1480/1480 - 34s - 23ms/step - binary_accuracy: 0.9982 - los

In [23]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s1.9_t1_w5_gt38/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.1_t48_w5_gt91/model.h5



Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.2_t68_w5_gt109/model.h5

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.2_t98_w5_gt127/model.h5
Done.

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s1.9_t1_w5_gt38/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/MitoSplit-Net/Models/multich_spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/history

Saving /mnt/LEB/Scientific_projects/deep_events_WS/data/single_channel_fluo/Mit